In [39]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

import methods


In [40]:
n = 10
columns = ['N', 'late_penality', 'waiting_charge', 'actual_time']
for i in range(1, n + 1):
    columns.append(f'a{i}')
df = pd.DataFrame(columns=columns)

In [41]:
for _ in range(10000):
    N = random.randint(n + 11, n + 39)
    intervals = np.random.gamma(shape=4, scale=2, size=N)

    # Variable gamma
    intervals = np.random.gamma(shape=random.randint(3, 7), scale=2, size=N)
    
    first_n = intervals[:n]
    late_penalty = random.randint(25, 500)
    waiting_charge = random.uniform(0.5, 1.8)
    actual_time = methods.cal_actual_time(N=N, n=5, intervals=intervals)
    dic = {'N': N, 'late_penality': late_penalty, 'waiting_charge': waiting_charge, 'actual_time': actual_time}
    for i in range(1, n + 1):
        dic[f'a{i}'] = first_n[i - 1]
    df.loc[len(df)] = dic
    # df.append(dic, ignore_index=True)


In [42]:
df.head()


,N,late_penality,waiting_charge,actual_time,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10
0,25,135,1.529240,295.038972,12.456914,21.612745,7.231214,13.474628,14.188965,22.499983,7.870810,16.425760,7.553698,14.957868
1,36,311,1.691124,282.130995,16.086311,3.932088,9.724915,3.613821,11.558840,8.497447,9.046592,12.219965,6.430300,5.894443
2,49,63,0.503842,350.307730,7.713647,4.568167,10.853264,5.635060,14.055548,4.278703,8.163972,7.579203,6.410769,13.534009
3,32,195,0.932197,291.373065,7.569350,6.462954,6.546044,17.136689,10.540773,9.748333,10.351257,10.628924,11.151028,15.981365
4,44,115,1.184295,386.090299,15.475840,13.165084,18.494866,4.920777,7.030130,4.047918,6.655596,5.910030,6.366335,10.561638


In [43]:
features = [x for x in columns if x != 'actual_time']
X = df[features]
y = df['actual_time']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
model = LinearRegression()

model.fit(X_train, y_train)
print(model.score(X_train, y_train))
model.coef_

0.8629268310845615


array([ 1.00420116e+01, -1.02040305e-04,  1.73231685e-02,  1.87144307e+00,
        2.00412078e+00,  1.97267867e+00,  1.97093527e+00,  2.06333167e+00,
        3.05265270e+00,  2.89320284e+00,  3.16118094e+00,  3.14171833e+00,
        2.79309649e+00])

In [45]:
print(model.score(X_test, y_test))

0.8610971837839004


In [46]:
gamma_shape = 4
gamma_scale = 2
N = 40
late_penalty = 100
waiting_charge = 1
costs = []
delays = []
for _ in range(10000):
    intervals = np.random.gamma(shape=4, scale=2, size=N)
    first_n = intervals[:n]
    dic = {'N': [N], 'late_penality': [late_penalty], 'waiting_charge': [waiting_charge]}
    for i in range(1, n + 1):
        dic[f'a{i}'] = [first_n[i - 1]]
    predicted_time = model.predict(pd.DataFrame(dic))[0] - 10
    actual_time = methods.cal_actual_time(N=N, n=n, intervals=intervals)
    cost = methods.cal_cost(late_fee=late_penalty, waiting_charge=waiting_charge, actual_time=actual_time, predicted_time=predicted_time)
    costs.append(cost)
    delays.append(actual_time - predicted_time)
methods.plot_plotly(data=costs, data_label='Cost')
methods.plot_plotly(data=delays, data_label='Delay')